In [1]:
import matplotlib.pyplot as plt, numpy as np, pandas as pd, importlib
import astropy.constants as const, astropy.units as u, astropy.table as tbl
import astrometric_coeffs as ac
import json

from uncertainties import unumpy as unp, ufloat
from uncertainties import correlated_values_norm, correlation_matrix

#from AMRF_auxil_routines import *
import warnings

from astropy.io import fits

----------------------------------------
#### Initialize

Read the table

In [2]:
# Where do I find the data?
# -------------------------
data_folder = r"../" 

# Retrieve the NSS table
# ----------------------
# NSS_file   = data_folder + 'xmatch_with_AMRF.fits'
# data       = tbl.Table.read(NSS_file)

# idxs = ((data['nss_solution_type'] == 'Orbital') | (data['nss_solution_type'] == 'AstroSpectroSB1'))
# data = data[idxs]
# data = tbl.Table.read('../xmatch_candidates.fits')
# data = tbl.Table(data[40])
# data['parsec_m1'] = 0.76
data = tbl.Table.read('../table_extra.fits')
# Get the AMRF limiting curves
limiting_curves = tbl.Table.read('../data/other/AMRF_limiting_curves.fits')

# data.remove_column('source_id')
# data.remove_column('source_id_2')
# data['source_id_1'].name = 'source_id'
# Last time we used particular bit indices... I dont think we need it now.
# (data['bit_index'] == 8191) | (data['bit_index'] == 65535)) & \

------------------------------------------------------
#### Calculate the AMRF, qmin, acceleration, and their errors

In [3]:
# Retrieve the conservative limiting AMRF values for some primary mass
# --------------
def Atr(m1):
    j = np.argmin(np.abs(m1 - limiting_curves['m1'].data))
    return limiting_curves['Atr'][j]
def Ams(m1):
    j = np.argmin(np.abs(m1 - limiting_curves['m1'].data))
    return limiting_curves['Ams'][j]

In [4]:
# =============================================================================
#                Auxil routines to obtain the covariance matrix
# =============================================================================

# 1) get the list of parameters from the solution type
def get_par_list(solution_type=None):
    if (solution_type is None) or (solution_type=='Orbital'):
        return ('ra', 'dec', 'parallax', 'pmra', 'pmdec', 'a_thiele_innes',
                'b_thiele_innes', 'f_thiele_innes', 'g_thiele_innes',
                'eccentricity', 'period', 't_periastron')

    elif (solution_type=='OrbitalAlternative') or (solution_type=='OrbitalAlternativeValidated') \
            or (solution_type=='OrbitalTargetedSearch') or (solution_type=='OrbitalTargetedSearchValidated'):
        return ('ra', 'dec', 'parallax', 'pmra', 'pmdec', 'a_thiele_innes',
                'b_thiele_innes', 'f_thiele_innes', 'g_thiele_innes',
                'period', 'eccentricity', 't_periastron')

    elif solution_type=='AstroSpectroSB1':
        return ('ra', 'dec', 'parallax', 'pmra', 'pmdec', 'a_thiele_innes',
                'b_thiele_innes', 'f_thiele_innes', 'g_thiele_innes',
                'c_thiele_innes', 'h_thiele_innes', 'center_of_mass_velocity',
                'eccentricity', 'period', 't_periastron')


# 2) Get the order of parameters in the covariance matrix, for a given bit index.
def bit_index_map(bit_index):
    if bit_index==8191:
        return ['ra','dec','parallax','pmra','pmdec','A','B','F','G', 'e','P', 'T']
    elif bit_index==8179:
        return ['ra','dec','parallax','pmra','pmdec','A','B','F','P', 'T']
    elif bit_index==65535:
        return ['ra', 'dec', 'parallax', 'pmra', 'pmdec', 'A', 'B', 'F', 'G', 'C', 'H', 'gamma','e', 'P', 'T']
    elif bit_index==65435:
        return ['ra', 'dec', 'parallax', 'pmra', 'pmdec', 'A', 'B', 'F', 'H', 'gamma', 'P', 'T']
    else:
        return None
    

# 3) Generate the correlation matrix
def make_corr_matrix(input_table, pars=None):
    """
    INPUT:
    input_table nss_two_body_orbit table.
    pars : list
            list of parameters for the corresponding solution of the desired
              target, in the same order as they appear in the Gaia table.
      """
    if pars is None:
        pars = get_par_list()

    # read the correlation vector
    s1 = input_table['corr_vec'].replace('\n','')   
    s1 = s1.replace(' ',',')
    s1 = s1.replace('--','0')
    corr_vec = list(json.loads(s1))
    # set the number of parameters in the table
    n_pars = len(pars)
    # define the correlation matrix.
    corr_mat = np.ones([n_pars, n_pars], dtype=float)

    # Read the matrix (lower triangle)
    ind = 0
    for i in range(n_pars):
        for j in range(i):
            corr_mat[j][i] = corr_vec[ind]
            corr_mat[i][j] = corr_vec[ind]
            ind += 1

    return corr_mat


# 4) Get the NSS data 
def get_nss_data(input_table, source_id):

    target_idx = np.argwhere(input_table['source_id'] == source_id)[0][0]
    pars = get_par_list(input_table['nss_solution_type'][target_idx])
    corr_mat = make_corr_matrix(input_table[target_idx], pars=pars)

    mu, std = np.zeros(len(pars)), np.zeros(len(pars))
    for i, par in enumerate(pars):
        try:
            mu[i] = input_table[par][target_idx]
            std[i] = input_table[par + '_error'][target_idx]
        except KeyError:
            mu[i], std[i] = np.nan, np.nan

    nan_idxs = np.argwhere(np.isnan(corr_mat))
    corr_mat[nan_idxs[:, 0], nan_idxs[:, 1]] = 0.0

    return mu, std, corr_mat

In [5]:
def multivar_sample(mu, sigma, corr, n):
    cov = corr*(sigma[:, None] * sigma[None, :])
    # l = spla.cholesky(cov)
    # z = np.random.normal(size=(n, mu.shape[0]))
    # return z.dot(l) + mu
    return np.random.multivariate_normal(mu, cov, size=n)

In [6]:
# =============================================================================
#                      calc parameters.
# =============================================================================
# Here we calculate the AMRF, qmin, etc, assuming that the mass of the luminous star 
# is exactly one solar mass. This is just for the red-clump stars...
def calc_AMRF(par_in, par_in_errors, corr_matrix, m1, bit_index=8191):
    """
    For the given set of orbital parameters by Gaia, this function calculates
    the standard geometrical elements (a, omega, Omega, and i). If the error
    estimates and covariance matrix are prodived, the error estimates on the
    calculated parameters are returned as well.

    Input: thiele_innes: Thiele Innes parameters [A,B,F,G] in milli-arcsec
           thiele_innes_errors : Corresponding errors.
           corr_matrix : Corresponding  4X4 correlation matrix.

  Output: class-III probability via monte carlo
    """
    # Read the coefficients and assign the correlation matrix.
    # Create correlated quantities. If the error is nan we assume 1e-6...
    par_in_errors[np.isnan(par_in_errors)] = 1e-6
    par_list = correlated_values_norm([(par_in[i], par_in_errors[i]) for i in np.arange(len(par_in))], corr_matrix)
    key_list = bit_index_map(bit_index)

    par = {key_list[i]: par_list[i] for i in np.arange(len(key_list))}
    par['mass'] = m1

    # Add the G Thiele-Innes parameter if needed.
    if (bit_index == 8179) | (bit_index == 65435):
        G = -par['A']*par['F']/par['B']
    else:
        G = par['G']

    # This in an intermediate step in the formulae...
    p = (par['A'] ** 2 + par['B'] ** 2 + G ** 2 + par['F'] ** 2) / 2.
    q = par['A'] * G - par['B'] * par['F']

    # Calculate the angular semimajor axis (already in mas)
    a_mas = unp.sqrt(p + unp.sqrt(p ** 2 - q ** 2))

    # Calculate the inclination and convert from radians to degrees
    i_deg = unp.arccos(q / (a_mas ** 2.)) * (180 / np.pi)
    
    try:
        if par.get("d") is not None:
            K_kms = 4.74372*unp.sqrt(par['C'] ** 2 + par['H'] ** 2)*(2*np.pi)/(par['P']/ 365.25)/np.sqrt(1-par['e']**2)
            acc   = 2*K_kms/par['P']
        else:
            K_kms = 4.74372*unp.sqrt(par['C'] ** 2 + par['H'] ** 2)*(2*np.pi)/(par['P']/ 365.25)
            acc   = K_kms/par['P']/4
    except:
        K_kms = ufloat(999, 999) 
        acc   = ufloat(999,999)

    # Calculate the AMRF
    try:
        AMRF = a_mas / par['parallax'] * par['mass'] ** (-1 / 3)  * (par['P']/ 365.25) ** (-2 / 3)

        # Calculate AMRF q
        y = AMRF ** 3
        h = (y/2 + (y**2)/3 + (y**3)/27
             + np.sqrt(3)/18*y*unp.sqrt(4*y+27))**(1/3)
        q = h + (2*y/3 + (y**2)/9)/h + y/3
    except:
        AMRF = ufloat(np.nan, np.inf) 
        q    = ufloat(np.nan, np.inf) 
        
    # Extract expectancy values and standard deviations
    pars = np.array([unp.nominal_values(AMRF),
                         unp.nominal_values(q),
                         unp.nominal_values(a_mas),
                         unp.nominal_values(i_deg),
                         unp.nominal_values(K_kms),
                         unp.nominal_values(acc)])

    pars_error = np.array([unp.std_devs(AMRF),
                               unp.std_devs(q),
                               unp.std_devs(a_mas),
                               unp.std_devs(i_deg),
                               unp.std_devs(K_kms),
                               unp.std_devs(acc)])

    return pars, pars_error

In [7]:
def class_probs(Atr,Ams,par_in, par_in_errors,
                  m1, m1_error, corr_matrix, bit_index=8191, n=1e2, factor=1.0):
    """
    For the given set of orbital parameters by Gaia, this function calculates
    the standard geometrical elements (a, omega, Omega, and i). If the error
    estimates and covariance matrix are prodived, the error estimates on the
    calculated parameters are returned as well.

    Input: thiele_innes: Thiele Innes parameters [A,B,F,G] in milli-arcsec
           thiele_innes_errors : Corresponding errors.
           corr_matrix : Corresponding  4X4 correlation matrix.

  Output: physical and geometrical parameters
    """
    r_3 = 0
    r_2 = 0
    par_in_errors[np.isnan(par_in_errors)] = 1e-6
    vecs = multivar_sample(par_in, par_in_errors, corr_matrix, int(n))
    key_list = bit_index_map(bit_index)

    for vec in vecs:
        par = {key_list[i]: vec[i] for i in np.arange(len(key_list))}
        par['mass'] = m1_error*np.random.randn() + m1

        # Add the G Thiele-Innes parameter if needed.
        if (bit_index == 8179) | (bit_index == 65435):
            par['G'] = -par['A'] * par['F'] / par['B']

        # This in an intermediate step in the formulae...
        p = (par['A'] ** 2 + par['B'] ** 2 + par['G'] ** 2 + par['F'] ** 2) / 2.
        q = par['A'] * par['G'] - par['B'] * par['F']

        # Calculate the semimajor axis (already in mas)
        a_mas = np.sqrt(p + np.sqrt(p ** 2 - q ** 2))

        # Calculate the AMRF
        AMRF = a_mas / par['parallax'] * par['mass'] ** (-1 / 3) * (par['P']/ 365.25) ** (-2 / 3)

        try:
            if 0 < par['e'] < 1:
                if AMRF > Atr * factor:
                    r_3 += 1
                elif Ams * factor < AMRF < Atr * factor:
                    r_2 += 1
        except KeyError:
            pass

    return (n-r_2-r_3)/n, r_2/n, r_3/n #(no_detections + detections)


In [8]:
# =============================================================================
#                       Read the data from the NSS table
# =============================================================================
def add_astrometric_parameters(data):
    from tqdm import tqdm
    # Here we only calculate (but don't assign class 3 probabilities!
    # We get the data table, arrange the arrays, calculate the astrometric
    # coefficients and plug it all back into the table.

    # Initialize the arrays
    # ---------------------
    # We need to calculate the AMRF, mass ratio, angular semi-major axis, orbtial inclination
    # and order-of-magnitude acceleration. We also want their uncertainties.
    count_good, count_bad = 0, 0
    A, q, a_mas, i_deg, K_kms, acc, P1, P2, P3 = np.full(len(data), np.nan),  np.full(len(data), np.nan), \
                              np.full(len(data), np.nan),  np.full(len(data), np.nan), \
                              np.full(len(data), np.nan), np.full(len(data), np.nan), \
                              np.full(len(data), np.nan), np.full(len(data), np.nan), np.full(len(data), np.nan) 

    Ae, qe, a_mase, i_dege, K_kmse, acce = np.full(len(data), np.nan),  np.full(len(data), np.nan), \
                                   np.full(len(data), np.nan),  np.full(len(data), np.nan), \
                                   np.full(len(data), np.nan),  np.full(len(data), np.nan)

    # Now go one by one and calculate the AMRF
    # ----------------------------------------
    for idx in tqdm(range(len(data['source_id']))):
        
        # Read the NSS solutin values.
        sid = data['source_id'][idx]
        mu, std, corr_mat = get_nss_data(data, sid)
        m1 = data['m1'][idx]
        m1_error = data['m1_err'][idx]
        if np.ma.is_masked(m1):
            print(idx)
            pass
        Ams_idx = data['Ams'][idx]
        Atr_idx = data['Atr'][idx]
        if np.ma.is_masked(Ams_idx) | np.ma.is_masked(Atr_idx):
            Ams_idx = Ams(m1)
            Atr_idx = Atr(m1)
            
        vals, stds = calc_AMRF(mu, std, corr_mat, m1, bit_index=data['bit_index'][idx])
        p1, p2, p3 = class_probs(data['Atr'][idx],data['Ams'][idx],mu, std, m1, m1_error, corr_mat, bit_index=data['bit_index'][idx], n = 1e4)
        try:
            A[idx], Ae[idx]  = vals[0], stds[0]
            q[idx], qe[idx]  = vals[1], stds[1]
            a_mas[idx], a_mase[idx]  = vals[2], stds[2]
            i_deg[idx], i_dege[idx]  = vals[3], stds[3]
            K_kms[idx], K_kmse[idx]  = vals[4], stds[4]
            acc[idx],   acce[idx]    = vals[5], stds[5]
            P1[idx], P2[idx], P3[idx] = p1, p2, p3
        except:
            pass
    
    # Store it all back in the original data structure.
    data['AMRF'], data['AMRF_error'] = A, Ae
    data['AMRF_q'], data['AMRF_q_error'] = q, qe
    data['a_mas'], data['a_mas_error'] = a_mas, a_mase
    data['i_deg'], data['i_deg_error'] = i_deg, i_dege
    data['K_kms'], data['K_kms_error'] = K_kms, K_kmse
    data['acc_kmsd'], data['acc_kmsd_error'] = acc, acce
    data['classI_prob'] = P1
    data['classII_prob'] = P2
    data['classIII_prob'] = P3
    return data


In [9]:
data_new = add_astrometric_parameters(data)

100%|██████████| 12/12 [00:00<00:00, 12.98it/s]


In [10]:
data_new['m2'] = data_new['m1'] * data_new['AMRF_q']
data_new['m2_err'] = ((data_new['m1_err'] * data_new['AMRF_q'])**2 + (data_new['m1'] * data_new['AMRF_q_error'])**2)**(1/2)
data_new['AMRF','AMRF_error','AMRF_q','AMRF_q_error','m1','m2','m2_err','classI_prob'].show_in_notebook()

idx,AMRF,AMRF_error,AMRF_q,AMRF_q_error,m1,m2,m2_err,classI_prob
0,0.28896428942003444,0.011522877727919354,0.3535922603933946,0.017073335990875703,1.0813108081428187,0.3823431328390275,0.025540267562753793,1.0
1,0.30802163587897463,0.011798028271333968,0.38220481868046646,0.017948085143336175,0.7996153884984311,0.3056168545751536,0.020013782895952743,0.5134
2,0.15724336133570369,0.011518305443310336,0.17510121930493,0.01424112830206775,1.1996726587740194,0.21006414531811804,0.023332632465867524,1.0
3,0.22110698403557016,0.014741722549614783,0.2576125632372684,0.019892139767786227,1.4055332091866153,0.3620830127336677,0.03224525004273336,0.9984
4,0.22032549337862437,0.007573665492626467,0.25655860376609496,0.010208761472711353,0.8411180223235422,0.21579606540982707,0.01512106405668068,0.998
5,0.2971382673333738,0.004878427477021414,0.3657722729215606,0.0073104963664453815,1.2145451755089711,0.4442469494118321,0.015601467273050474,1.0
6,0.5181400987927655,0.02078972329055196,0.7534216927264712,0.04236623745258607,1.4837378713408065,1.1178802985879615,0.10112565262427102,0.0619
7,0.3593227778657604,0.008665816991490459,0.4630900543700287,0.014155289189041277,0.9930278482124181,0.4598613202196413,0.026900124107372494,0.9916
8,0.30220012935011675,0.009403531923441786,0.37338427756723835,0.014190571742933929,1.7112012271029515,0.6389356339540073,0.03571777099585102,0.0038
9,0.35414447892042716,0.006259824531824189,0.4546618415685146,0.010151914447142328,1.114009276403983,0.5064975091342433,0.02798623425030718,0.2111


In [11]:
nn_data = data_new[data_new['classI_prob']<0.1]
nn_data = nn_data[nn_data['probability']>0.99]
# nn_data = nn_data[~np.isnan(nn_data['AM'])]
nn_data[['source_id','idx','ra','dec','AMRF','m1','m2','probability','classI_prob','classIII_prob']]

source_id,idx,ra,dec,AMRF,m1,m2,probability,classI_prob,classIII_prob
,,deg,deg,,,,,,
int64,int64,float64,float64,float64,float64,float64,float64,float64,float64
181021811989304704,93,79.45472711306492,32.66819550267403,0.5181400987927655,1.4837378713408065,1.1178802985879615,1.0,0.0619,0.9344
4474102036798006912,96,266.57868890514794,5.9353026539963585,0.30220012935011675,1.7112012271029515,0.6389356339540073,1.0,0.0038,0.0
395159947842384000,142,4.27217464818377,52.27818573722861,0.44599646700823137,2.080912280290238,1.2767558450415486,1.0,0.0,0.7845


In [12]:
nn_data.write('../table_extra.fits', format='fits', overwrite=True)